In [1]:
!pip install langchain-community         

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66
  Attempting uninstall: langchain-text-

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 118.5 MB/s  0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.document_loaders.csv_loader import CSVLoader   
from langchain_community.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS    
from charset_normalizer import from_path
import pandas as pd 
from langchain_community.embeddings import HuggingFaceEmbeddings  
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline  
from langchain_community.llms import HuggingFacePipeline  

2025-08-29 16:43:08.671842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756485789.016672      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756485789.116163      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
csv_path = "/kaggle/input/qa-dataset/codebasics_faqs.csv" 
  
# 1) Detect encoding
enc = (from_path(csv_path).best() or from_path(csv_path).first()).encoding or "latin1" 

# 2) Read CSV (handle both new/old pandas) 
read_kwargs = {"encoding": enc} 
try:
    df = pd.read_csv(csv_path, on_bad_lines="skip", **read_kwargs)  
except TypeError:  
    # for older pandas 
    df = pd.read_csv(csv_path, error_bad_lines=False, **read_kwargs) 
   
# 3) Turn into Documents
try:  
    from langchain_community.document_loaders import DataFrameLoader 
except ImportError:  
    # Fallback if langchain_community isn't available
    from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="prompt")  # <- use your text column 
docs = loader.load() 
 
docs[:2]  # quick peek  

[Document(metadata={'response': 'Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.'}, page_content='I have never done programming in my life. Can I take this bootcamp?'),
 Document(metadata={'response': 'Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.'}, page_content='Why should I trust Codebasics?')]

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
vectordb = FAISS.from_documents(documents=docs, embedding=embeddings)       

/tmp/ipykernel_19/2742844143.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]